In [1]:
import numpy as np
from numpy import sin,cos,pi,exp
from numpy.linalg import norm
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Lattice, Structure
import matplotlib.pyplot as plt
from pymatgen.vis.structure_vtk import StructureVis
from IPython.display import display
%matplotlib widget

def to_dict(struct): # changes structure to the kind of dict suitable for Electric Field
    PMG_struct_dict = struct.as_dict()
    struct_dict = {}
    for atom in PMG_struct_dict['sites']:
        atom_name = atom['species'][0]['element']
        try:
            struct_dict[atom_name].append(np.array(atom['xyz']))
        except:
            struct_dict[atom_name] = [np.array(atom['xyz'])]
    return struct_dict

def lattice_parameters_cartesian(structure):
    ABC = structure.lattice.__dict__["_matrix"]
    return ABC[0], ABC[1], ABC[2]

def find_bonds(atom_index, struct, remove_self=False, visual=False):
    """
    finds the atoms (OXYGENS) bonded to a specific site, and returns a Structure containing with only the ion and its neighbors
    """
    num_atoms = len(struct)
    filled_struct = struct.copy().make_supercell(3,3,3)
    atom_index = 27*atom_index + 13
    bonded_atoms = [] if remove_self else [filled_struct[atom_index]]
    atom_symbol = filled_struct[atom_index].specie.symbol
    for i in range(len(filled_struct)):
        crystal_atom_symbol = filled_struct[i].specie.symbol
        if crystal_atom_symbol == "O": #True
            max_d = 3 #CovalentRadius.radius[atom_symbol] + CovalentRadius.radius[crystal_atom_symbol] + 0.2 # 0.2 bond tolerance
            d = filled_struct.get_distance(atom_index, i, np.array([0, 0, 0]))
            if d < max_d:
                bonded_atoms.append(filled_struct[i])
    output = Structure.from_sites(bonded_atoms)
    if visual:
        visualize_struct(output)
    return output

def electric_field(r,lattice_point_dict,charge_dict): # r is the position vector where you want to know the total field
    """
    Finds and returns the electric field at a specified point based on the lattice points
    
    r - the position vector where you want to know the total field
    lattice_point_dict - must be a dictionary - if generated from a structure use to_dict function
    charge_dict - must name each atom present, and it's ionic charge
    """
    field = np.array((0, 0, 0))

    for element in lattice_point_dict.keys():
        # if element == 'O':
        #     pass
        # else:
        for R in lattice_point_dict[element]:  # R is a position vector for the ion producing the field
            d = r - R
            if norm(d) < 0.1:
                pass
            else:
                field = field + charge_dict[element]*d/(norm(d)**3)
    return field

def visualize_struct(struct, polarization={}, add_bond_center=[],
                     add_bond_neighbors=[], field_strength=[],
                     ind=0, relative_vectors=[], show_bonds=False):
    """
    Visualizes structure in seporate window - have to close window to continue running other code
    
    struct - pymatgen Structure
    field_strength - list of strengths of fields that will appear as lines in the image
    ind - the index of the atom in struct that you want the lines to be based on
    relative_vectors - adds lines that start at the terminal points of the field_strenth lines
    """
    vis = StructureVis(element_color_mapping={'Eu': (0, 255,200), 'O':(255,0,0), 'Y':(100, 100, 255), 'Si':(255, 255, 50)}, show_bonds=show_bonds)
    # Eu3+ : Cyan, O2- : Red, Y3+ : blue, Si4+ : Yellow 
    vis.show_polyhedron = False
    vis.poly_radii_tol_factor -= 0.25
    vis.set_structure(struct.copy())
    if type(field_strength) == dict:
        e_color = {'O':(225,0,0), 'Y':(0,0,255),  'Si':(0,255,0), 'Eu':(200,255, 0), 'Be':(0,0,0)}
        for element in field_strength.keys():
            for i in range(len(field_strength[element])):
                vis.add_line(struct.cart_coords[ind], struct.cart_coords[ind] + field_strength[element][i]*5, e_color[element], width=10)
    else:
        for i in range(len(field_strength)):
            vis.add_line(struct.cart_coords[ind], struct.cart_coords[ind] + field_strength[i], (0, 255, 0), width=10)
            for j in range(len(relative_vectors)):
                vis.add_line(struct.cart_coords[ind] + field_strength[i], struct.cart_coords[ind] + field_strength[i] + relative_vectors[j], (225, 0, 0), width=10)
    for D_ in polarization:
        vis.add_line((0, 0, 0), polarization[D_], (0,0,0))
        vis.add_text(polarization[D_], D_, (0,0,0))
      
    for idx, center in enumerate(add_bond_center):
        vis.add_bonds(add_bond_neighbors[idx], center)
    
    return display(vis.show())

In [2]:
structure = Structure.from_file("C:/Users/onix/Downloads/Y2SiO5-mp3520.cif").to_conventional().sort()

# setting the perameters to what they apear to be from the code
beta = 122.18778465
a = np.array([1,0,0])*14.43523963
b = np.array([0,1,0])*6.74280580
c = np.array([cos(beta*np.pi/180),cos(90*np.pi/180),sin(beta*np.pi/180)])*10.42107312

angle_D1=11.35 *np.pi/180
angle_D2=(11.35 + 90) *np.pi/180
D1_rotation = np.array([[cos(angle_D1), 0, sin(angle_D1)], [0, 1, 0], [-sin(angle_D1), 0, cos(angle_D1)]])
D2_rotation = np.array([[cos(angle_D2), 0, sin(angle_D2)], [0, 1, 0], [-sin(angle_D2), 0, cos(angle_D2)]])

D1 = np.matmul(D1_rotation, c)
D2 = np.matmul(D2_rotation, c)
structure._lattice = Lattice([a, b, c])
crystal = structure.copy().sort()

lattice_points = {'Y3+': [], 'Si4+': [], 'O2-': []}
for i in range(len(crystal)):
       name = crystal[i].species_string
       lattice_points[name].append(crystal.cart_coords[i])

In [3]:
charges = {'Y': 3, 'Si': 4, 'O': -2, 'Eu': 3}
K = 8.99*10**9 * (1.6*10**(-19)) / 10**(-20)     # K = (1/4 pi epsilon_0) * e / (1 angstrom)**2  
K_gig = K*10**(-9) # Giga K units
cell_vector_struct = structure.copy().append("Li", np.array([1,0,0])).append("Li", np.array([0,1,0])).append("Li", np.array([0,0,1])) # To get transformed Cartesian unit vectors
a,b,c = cell_vector_struct.cart_coords[-3:]

In [4]:
field_at_pts = []
for i in range(16):
    point_field = electric_field(find_bonds(i, crystal)[0].coords, to_dict(find_bonds(i, crystal,visual=False)), charges)
    b_proj = np.dot(b, point_field)/(norm(point_field)*norm(b))*b
    ac_proj = point_field-b_proj
    D2_proj =  np.dot(D2, ac_proj)/(norm(ac_proj)*norm(D2))*D2
    print(f"{norm(point_field)*K:.3e}   \
    φ = {np.arccos(norm(b_proj)/norm(b))*180/np.pi:.3e}   \
    θ = {np.arccos(norm(D2_proj)/norm(D2))*180/np.pi:.3e} ")
    # θ, φ are the angles between the plane and the b and D2 axes respectively 
    field_at_pts.append(point_field*K_gig)

# Find bonds of Si4+
#centers = lattice_points['Si4+']
#oxygen = np.array(lattice_points['O2-'])
#print(structure.cart_coords)
#print(centers)
#neighborhoods = []
#for center in centers:
#    vec_to_center = oxygen - center
#    dist_to_center = np.linalg.norm(vec_to_center,axis=1)
#    neighbors = oxygen[np.argpartition(dist_to_center, 4)][0:4]
#    neighbors = Structure(neighbors, species=['O2-', 'O2-', 'O2-', 'O2-'])
#    neighborhoods.append(neighbors)

2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.433e+10       φ = 3.549e+01       θ = 8.932e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 
2.184e+10       φ = 3.329e+01       θ = 8.916e+01 


In [7]:
from pymatgen.core.periodic_table import Species
from pymatgen.core.sites import Site

EuYSO = structure.copy()
for i in range(4):
    EuYSO.replace(i, Species('Eu3+'))

polarization={"D1": D1, "D2": D2}
print(D1)
print(D2)

'''
NOTE: it is currently impossible to see D1, D2 axis and bonds at the same time due to some bug (not sure if it is us or package).
A hacky work around is to edit the source code directly. Go to pymatgen/vis/structure_vtk.py in the pymatgen package folder.
The path may look something like "C:/Users/onix/.venv/vlab/Lib/site-packages/pymatgen/vis/structure_vtk.py". On line 229
(in the set_structure method), copy paste the following lines of code (where you replace <D1> with the actual value of the D1 variable):

polarization={"D1": <D1>, "D2": <D2>} 
for D_ in polarization:
    self.add_line((0, 0, 0), polarization[D_], (0,0,0))
    self.add_text(polarization[D_], D_, (0,0,0))

Also note that you may have to press the show bonds button multiple times for it to appear.
'''

visualize_struct(EuYSO, polarization=polarization)

[-3.70701926e+00  6.38106692e-16  9.73944419e+00]
[9.73944419e+00 6.38106692e-16 3.70701926e+00]


AttributeError: 'NoneType' object has no attribute 'parent'

AttributeError: 'NoneType' object has no attribute 'parent'

AttributeError: 'NoneType' object has no attribute 'parent'

AttributeError: 'NoneType' object has no attribute 'parent'

...

AttributeError: 'NoneType' object has no attribute 'parent'

AttributeError: 'NoneType' object has no attribute 'parent'

AttributeError: 'NoneType' object has no attribute 'parent'

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

None